In [253]:


import cv2
import numpy as np
np.random.seed(42)


class driveFile:
  def __init__(self):
    self.path=""
  def dataload(self,folder_path):
    img=[]
    label=[]
    import os
    self.path1=os.path.join(self.path,folder_path)
    
    if os.path.exists(self.path1):
      for i in os.listdir(self.path1):
        self.dis_path=os.path.join(self.path1,i)
        # /content/drive/MyDrive/train_set/BA- cellulitis
        if os.path.exists(self.dis_path) and '.DS_Store' not in self.dis_path:
          for j in os.listdir(self.dis_path):
           if j.endswith(('.jpg', '.png', '.jpeg')):
              self.img_path = os.path.join(self.dis_path, j)
              loaded_img= self.imgload(self.img_path)
              if loaded_img is not None:
                img.append(loaded_img)
                label.append(i)
              else:
                 print("Skipping non-image file:", j)
          else:
              print("Skipping invalid path or .DS_Store:", self.dis_path)
              continue
      return np.array(img), np.array(label)
    else:
      return "patherror","patherror"



  def imgload(self,imgpath):
    img=cv2.imread(imgpath)
    if img is None:
      print(f"Failed to load image at {imgpath}")
      return None
    img=cv2.resize(img,(20,20))
    # v imp, usually 224*224 but here 20*20 "downsampling or using a smaller image resolution before flattening it."
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=img.astype('float32')/255.0
    
    img_array=np.array(img)
    return img_array

In [254]:

drive=driveFile()
images,labels=drive.dataload('dataset/train_set')
test_images,test_labels=drive.dataload('dataset/test_set')
images = (images - np.mean(images)) / np.std(images)
test_images = (test_images - np.mean(test_images)) / np.std(test_images)


Skipping invalid path or .DS_Store: dataset/train_set/FU-athlete-foot
Skipping invalid path or .DS_Store: dataset/train_set/VI-chickenpox
Skipping invalid path or .DS_Store: dataset/train_set/VI-shingles
Skipping invalid path or .DS_Store: dataset/train_set/FU-nail-fungus
Skipping invalid path or .DS_Store: dataset/train_set/BA-impetigo
Skipping invalid path or .DS_Store: dataset/train_set/PA-cutaneous-larva-migrans
Skipping invalid path or .DS_Store: dataset/train_set/FU-ringworm
Skipping invalid path or .DS_Store: dataset/train_set/BA- cellulitis
Skipping invalid path or .DS_Store: dataset/test_set/FU-athlete-foot
Skipping invalid path or .DS_Store: dataset/test_set/VI-chickenpox
Skipping invalid path or .DS_Store: dataset/test_set/VI-shingles
Skipping invalid path or .DS_Store: dataset/test_set/FU-nail-fungus
Skipping invalid path or .DS_Store: dataset/test_set/BA-impetigo
Skipping invalid path or .DS_Store: dataset/test_set/PA-cutaneous-larva-migrans
Skipping invalid path or .DS_St

In [255]:
images=np.array([img.flatten() for img in images])
test_images=np.array([img.flatten() for img in test_images])
labels=np.array(labels)
test_labels=np.array(test_labels)

unique_labels = np.unique(labels)
label_to_index = {label: idx for idx, label in enumerate(unique_labels)}

integer_encoded = np.array([label_to_index[label] for label in labels])
test_integer_encoded = np.array([label_to_index[label] for label in test_labels])
one_hot_encoded = np.eye(len(unique_labels))[integer_encoded]
test_one_hot_encoded = np.eye(len(unique_labels))[test_integer_encoded]
labels=one_hot_encoded
test_labels=test_one_hot_encoded


In [256]:
# benchmarking/sanity checking
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
labels_int = np.argmax(labels, axis=1)
test_labels_int = np.argmax(test_labels, axis=1)

scaler = StandardScaler()
images_scaled = scaler.fit_transform(images)
test_images_scaled = scaler.transform(test_images)

# Initialize and train SVM
svm_model = SVC(kernel='linear', random_state=42)  # Try 'rbf' kernel too for non-linear data
svm_model.fit(images_scaled, labels_int)

# Predictions and evaluation
svm_predictions = svm_model.predict(test_images_scaled)
svm_accuracy = accuracy_score(test_labels_int, svm_predictions)

print(f"SVM Accuracy: {svm_accuracy * 100:.2f}%")
print("Classification Report:")
print(classification_report(test_labels_int, svm_predictions))


SVM Accuracy: 67.81%
Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.48      0.54        33
           1       0.59      0.85      0.69        20
           2       0.83      0.75      0.79        32
           3       0.47      0.24      0.32        33
           4       0.50      0.78      0.61        23
           5       0.68      0.68      0.68        25
           6       0.84      0.91      0.87        34
           7       0.79      0.82      0.81        33

    accuracy                           0.68       233
   macro avg       0.66      0.69      0.66       233
weighted avg       0.67      0.68      0.66       233



In [257]:
class Layer:
    def __init__(self):
        self.input=None
        self.output=None
    def forward(self,input):
        # to be overridden 
        pass 
    def backward(self,output_gradient,learning_rate):
        pass
    

In [258]:
class Dense(Layer):
    def __init__(self,input_size,output_size):
        self.weights = np.random.randn(input_size, output_size) * np.sqrt(2 / input_size)
        self.bias = np.zeros((1, output_size))
        self.velocities_W=np.zeros_like(self.weights)
        self.velocities_B=np.zeros_like(self.bias)
        
    def forward(self,input):
        self.input=input
        return np.dot(self.input,self.weights)+self.bias
    
    def backward(self,output_gradient,learning_rate,momentcoeff):
        w_gradient = np.dot( self.input.T,output_gradient)  # (output_size, input_size)
        b_gradient = np.sum(output_gradient, axis=0, keepdims=True)  # (output_size, 1).T if axis=0 else axis=1 output_size, 1
        input_gradient = np.dot(output_gradient,self.weights.T)# (input_size, batch_size)
        self.velocities_W=  momentcoeff*self.velocities_W+(1- momentcoeff)* w_gradient
        self.velocities_B=  momentcoeff*self.velocities_B +(1- momentcoeff)* b_gradient
        self.weights-=(learning_rate*self.velocities_W)
        self.bias-=(learning_rate*self.velocities_B)
        return input_gradient

In [259]:
class Activation(Layer):
    def __init__(self,activation,activatn_derivative):
        #activation is a variable pointing to a activation method,for ex sigmoid
        self.activation=activation
        self.activatn_derivative=activatn_derivative
        
    def forward(self,input):
        self.input=input
        return self.activation(self.input)
    
    def backward(self,output_gradient,learning_rate,momentcoeff):
        return self.activatn_derivative(self.input)

In [260]:
class Sigmoid(Activation):
    def __init__(self):
        def sigmoid(x):
            return 1/(1+np.exp(-x))
        def sigmoid_derivative(x):
            s=sigmoid(x)
            return s*(1-s)
        super().__init__(sigmoid,sigmoid_derivative)
        

In [261]:
class Tanh(Activation):
    def __init__(self):
        def tanh(x):
            return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
        def tanh_derivative(x):
            t=tanh(x)
            return 1-(t**2)
        super().__init__(tanh,tanh_derivative)



In [262]:
class Relu(Activation):
    def __init__(self, alpha=0.1):
        self.alpha = alpha
        
        def prelu(x):
            return np.where(x > 0, x, self.alpha * x)
        
        def prelu_derivative(x):
            return np.where(x > 0, 1, self.alpha)
        
        super().__init__(prelu, prelu_derivative)


In [263]:
class Softmax:
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(input_size, output_size) * np.sqrt(2 / input_size)
        self.bias = np.zeros((1, output_size))
        self.velocities_W = np.zeros_like(self.weights)
        self.velocities_B = np.zeros_like(self.bias)
        
    def forward(self, input):
        self.input = input
        # Compute logits
        logits = np.dot(self.input, self.weights) + self.bias
        # Numerically stable softmax
        logits_shifted = logits - np.max(logits, axis=1, keepdims=True)
        exp_values = np.exp(logits_shifted)
        self.softmax = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        return self.softmax
    
    def backward(self, output_gradient, learning_rate, momentcoeff):
        # Compute gradients
        w_gradient = np.dot(self.input.T, output_gradient) / self.input.shape[0]
        b_gradient = np.sum(output_gradient, axis=0, keepdims=True)
        input_gradient = np.dot(output_gradient, self.weights.T)

        # Update weights and biases using momentum
        self.velocities_W=  momentcoeff*self.velocities_W+(1- momentcoeff)* w_gradient
        self.velocities_B=  momentcoeff*self.velocities_B +(1- momentcoeff)* b_gradient
        self.weights-=(learning_rate*self.velocities_W)
        self.bias-=(learning_rate*self.velocities_B)
        return input_gradient

In [264]:
class Loss(Layer):
    def __init__(self,loss_fn,loss_fn_Derivative):
        self.loss_fn=loss_fn
        self.loss_fn_Derivative=loss_fn_Derivative
    def forward(self,y_pred,y_true):
        self.y_pred = y_pred
        self.y_true = y_true
        return self.loss_fn(y_pred, y_true)
    def backward(self,y_pred,y_true):
        return self.loss_fn_Derivative(y_pred,y_true)

In [265]:
class BinaryCrossEntropy(Loss):
    def __init__(self):
        def bce(y_pred,y_true):
            y_pred = np.clip(y_pred, 1e-10, 1 - 1e-10)  # Prevent extreme values
            return -np.mean((y_true*np.log(y_pred))+((1-y_true)*np.log(1-y_pred)))
        def bce_derivative(y_pred,y_true):
            y_pred = np.clip(y_pred, 1e-10, 1 - 1e-10)  # Avoid division by zero
            return -(y_true / y_pred) + ((1 - y_true) / (1 - y_pred))
        super().__init__(bce,bce_derivative)


In [266]:
class SparseCategoricalCE(Loss):
    def __init__(self):
        def sce(logits,true_labels):
            exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True)) 
            probabilities = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)
            true_class_probs = probabilities[np.arange(len(true_labels)), true_labels]
            log_loss = -np.log(true_class_probs + 1e-15)  # Adding a small constant for numerical stability
            return np.mean(log_loss)
        def sce_derivative(logits, true_labels):
            exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))  # Stability trick
            probabilities = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)
            batch_size = logits.shape[0]
            probabilities[np.arange(batch_size), true_labels] -= 1
            gradient = probabilities / batch_size
            
            return gradient
        super().__init__(sce,sce_derivative)
    

In [267]:

class CategoricalCrossEntropyLoss(Loss):
    def __init__(self):
        def cel(y_predSM,y_true):
            # direct correspondance of values, element wise multi
            return -np.mean(np.sum(y_true * np.log(y_predSM+ 1e-9), axis=1))
        def cel_derivative(y_predSM,y_true):
            return y_predSM-y_true
        # der wrt SM = der wrt logit(z)
        super().__init__(cel,cel_derivative)

In [268]:


class Model:
    def train(self,images,labels,layers,hyperpara):
        
        for key, value in hyperpara.items():
            setattr(self, key, value)
        self.loss_fn=CategoricalCrossEntropyLoss()
        self.X=images
        self.y=labels
        self.layers=layers
        self.softmax=Softmax(self.neurons[-1],self.softmax_neurons)
        
        for epoch in range(self.epochs):
            num_samples = self.X.shape[0]
            # index=np.arange(num_samples)
            # np.random.shuffle(index)
            # self.X=self.X[index]
            # self.y=self.y[index]
            epoch_loss=0
            for i in range(0, num_samples, self.batch_size):
                if(i+self.batch_size==num_samples):
                    X_batch = self.X[i:num_samples]
                    y_batch = self.y[i:num_samples]
                else:
                    X_batch = self.X[i:i + self.batch_size]
                    y_batch = self.y[i:i + self.batch_size]
                
                ActiPredicted=self.forward(X_batch)
                
                pred_y=self.softmax.forward(ActiPredicted)
                
                loss=self.loss_fn.forward(pred_y,y_batch)
                epoch_loss += loss
                loss_gradient=self.loss_fn.backward(pred_y,y_batch)
                # print(f"Epoch {epoch + 1}, Batch {i // self.batch_size + 1}, Loss: {loss:.4f}")

                # Backward pass
                loss_gradient=self.softmax.backward(loss_gradient,self.learning_rate,self.momentum_coeff)
                self.backward(loss_gradient)
            print(f"Epoch {epoch + 1}, Average Loss: {epoch_loss / (num_samples / self.batch_size):.4f}")
                
                
                    
                
    def forward(self,X):
        for layer in self.layers:
            # here layer is obj to class not class*
            X=layer.forward(X)
        return X
        

    def backward(self, loss_gradient):
        for layer in reversed(self.layers):
            loss_gradient = layer.backward(loss_gradient, self.learning_rate,self.momentum_coeff)
    def predict(self, X):
        logits = self.forward(X)
        probabilities = self.softmax.forward(logits)
        return np.argmax(probabilities, axis=1)

    def evaluate(self, X, y):
        predictions = self.predict(X)
        true_labels = np.argmax(y, axis=1)
        accuracy = np.mean(predictions == true_labels)
        print(f"Accuracy: {accuracy * 100:.2f}%")
        return accuracy

        

In [269]:
hyperparameters={
    "hiddenLayers":3,
    "learning_rate":0.00001,
    "batch_size":32,
    "epochs":150,
    "ActivationFns":None,
    "neurons":None,
    "softmax_neurons":labels.shape[1],
    "momentum_coeff":0.9
    # v=coeff*v-learnrate*gradient
    # w=w+v
    
}

# change
hyperparameters["neurons"]=[32,32,16]
hyperparameters["ActivationFns"]=[Relu(),Relu(),Relu()]
layers=[]
inputsize=images.shape[1]
for i in range(hyperparameters["hiddenLayers"]):
    outputsize=hyperparameters["neurons"][i]
    layers.extend([Dense(inputsize,outputsize),hyperparameters["ActivationFns"][i]])
    inputsize=outputsize


model=Model()
model.train(images,labels,layers,hyperparameters)

Epoch 1, Average Loss: 3.1205
Epoch 2, Average Loss: 2.8277
Epoch 3, Average Loss: 2.6148
Epoch 4, Average Loss: 2.4819
Epoch 5, Average Loss: 2.3977
Epoch 6, Average Loss: 2.3411
Epoch 7, Average Loss: 2.2985
Epoch 8, Average Loss: 2.2662
Epoch 9, Average Loss: 2.2416
Epoch 10, Average Loss: 2.2221
Epoch 11, Average Loss: 2.2059
Epoch 12, Average Loss: 2.1919
Epoch 13, Average Loss: 2.1802
Epoch 14, Average Loss: 2.1695
Epoch 15, Average Loss: 2.1601
Epoch 16, Average Loss: 2.1516
Epoch 17, Average Loss: 2.1445
Epoch 18, Average Loss: 2.1381
Epoch 19, Average Loss: 2.1323
Epoch 20, Average Loss: 2.1274
Epoch 21, Average Loss: 2.1231
Epoch 22, Average Loss: 2.1192
Epoch 23, Average Loss: 2.1158
Epoch 24, Average Loss: 2.1128
Epoch 25, Average Loss: 2.1101
Epoch 26, Average Loss: 2.1076
Epoch 27, Average Loss: 2.1055
Epoch 28, Average Loss: 2.1035
Epoch 29, Average Loss: 2.1018
Epoch 30, Average Loss: 2.1003
Epoch 31, Average Loss: 2.0988
Epoch 32, Average Loss: 2.0974
Epoch 33, Average

In [270]:
print(f"accuary:{model.evaluate(test_images,test_labels)}")

Accuracy: 17.60%
accuary:0.1759656652360515
